In [22]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types
import pandas as pd
import pyspark.sql.functions as F
from pyspark.sql.functions import isnan, when, count, col

In [24]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

print(type(spark))

    

<class 'pyspark.sql.session.SparkSession'>


In [3]:
    
df_green = spark.read \
    .option("header", "true") \
    .csv('/home/melse/traffic_project/data_ingestion/data/traffic-index-in-saudi-arabia-and-middle-east/traffic_index.csv')
    

In [4]:
df = pd.read_csv('/home/melse/traffic_project/data_ingestion/data/traffic-index-in-saudi-arabia-and-middle-east/traffic_index.csv')
print(df.dtypes)


City                    object
Datetime                object
TrafficIndexLive         int64
JamsCount                int64
JamsDelay              float64
JamsLength             float64
TrafficIndexWeekAgo      int64
TravelTimeHistoric     float64
TravelTimeLive         float64
dtype: object


In [5]:
schema = spark.createDataFrame(df).schema
print(schema)

/home/melse/spark/spark-3.3.4-bin-hadoop3-scala2.13/python/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/melse/spark/spark-3.3.4-bin-hadoop3-scala2.13/python/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


StructType([StructField('City', StringType(), True), StructField('Datetime', StringType(), True), StructField('TrafficIndexLive', LongType(), True), StructField('JamsCount', LongType(), True), StructField('JamsDelay', DoubleType(), True), StructField('JamsLength', DoubleType(), True), StructField('TrafficIndexWeekAgo', LongType(), True), StructField('TravelTimeHistoric', DoubleType(), True), StructField('TravelTimeLive', DoubleType(), True)])


In [6]:
new_schema = types.StructType([types.StructField('City', types.StringType(), True),
                         types.StructField('Datetime', types.TimestampType(), True),
                         types.StructField('TrafficIndexLive', types.IntegerType(), True),
                         types.StructField('JamsCount', types.IntegerType(), True),
                         types.StructField('JamsDelay', types.FloatType(), True),
                         types.StructField('JamsLength', types.FloatType(), True),
                         types.StructField('TrafficIndexWeekAgo', types.IntegerType(), True),
                         types.StructField('TravelTimeHistoric', types.FloatType(), True),
                         types.StructField('TravelTimeLive', types.FloatType(), True)])

In [7]:
new_df = spark.read \
    .option("header", "true") \
    .schema(new_schema) \
    .csv('/home/melse/traffic_project/data_ingestion/data/traffic-index-in-saudi-arabia-and-middle-east/traffic_index.csv')
    

In [8]:
new_df.printSchema()

root
 |-- City: string (nullable = true)
 |-- Datetime: timestamp (nullable = true)
 |-- TrafficIndexLive: integer (nullable = true)
 |-- JamsCount: integer (nullable = true)
 |-- JamsDelay: float (nullable = true)
 |-- JamsLength: float (nullable = true)
 |-- TrafficIndexWeekAgo: integer (nullable = true)
 |-- TravelTimeHistoric: float (nullable = true)
 |-- TravelTimeLive: float (nullable = true)



In [9]:
new_df.show()

+--------+-------------------+----------------+---------+---------+----------+-------------------+------------------+--------------+
|    City|           Datetime|TrafficIndexLive|JamsCount|JamsDelay|JamsLength|TrafficIndexWeekAgo|TravelTimeHistoric|TravelTimeLive|
+--------+-------------------+----------------+---------+---------+----------+-------------------+------------------+--------------+
|Abudhabi|2023-07-07 08:01:30|               6|        4|     15.6|       0.7|                 13|          59.61192|     54.803616|
|Abudhabi|2023-07-07 09:01:30|               7|        7|     20.5|       1.7|                  8|         60.221386|      56.11863|
|Abudhabi|2023-07-07 10:46:30|               7|        8|     25.0|       2.8|                  6|          59.16198|     55.518833|
|Abudhabi|2023-07-07 11:16:30|               8|       11|     30.6|       5.5|                  6|         59.738136|     56.413918|
|Abudhabi|2023-07-07 12:01:30|               8|       20|     62.1|  

In [11]:
new_df.columns

['City',
 'Datetime',
 'TrafficIndexLive',
 'JamsCount',
 'JamsDelay',
 'JamsLength',
 'TrafficIndexWeekAgo',
 'TravelTimeHistoric',
 'TravelTimeLive']

In [21]:
columns = ['City',
     'TrafficIndexLive',
     'JamsCount',
     'JamsDelay',
     'JamsLength',
     'TrafficIndexWeekAgo',
     'TravelTimeHistoric',
     'TravelTimeLive']

new_df.select([count(when(isnan(c), c)).alias(c) for c in columns]).show()

+----+----------------+---------+---------+----------+-------------------+------------------+--------------+
|City|TrafficIndexLive|JamsCount|JamsDelay|JamsLength|TrafficIndexWeekAgo|TravelTimeHistoric|TravelTimeLive|
+----+----------------+---------+---------+----------+-------------------+------------------+--------------+
|   0|               0|        0|        0|         0|                  0|                 0|             0|
+----+----------------+---------+---------+----------+-------------------+------------------+--------------+

